# Homework 2

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (A): US Treasuries in November 2008

## Data
* Use the data file `treasury_ts_2015-08-15.xlsx`.
* Examine the treasure issues with `kytreasno` of `204046` and `204047`. These are the bond and note (respectively) which mature on 2015-08-15.
* Look at the data on 2008-11-04.

## 1.1 The situation

Make a chart comparing the issues in the following features, (as of Nov 4, 2008.)
* coupon rate
* bid
* ask
* accrued interest
* dirty price
* duration (quoted in years, not days, assuming 365.25 days per year.)
* modified duration
* YTM

## 1.2 Hedge Ratio

Suppose you are building a trade to go long $n_i$ bonds (`204046`) and short $n_j$ notes (`204047`).

We can find an equation for $n_j$ in terms of $n_i$ such that the total holdings will have duration equal to zero. (Having zero duration also means have zero dollar duration, if helpful.)

Notation:
* $n_i$: number of bonds purchased (or sold)
* $D_i$: duration of bond $i$
* $D_{\$,i}$: dollar duration of bond $i$, equal to $p_iD_i$

If we want the total duration of our holdings to be zero, then we need to size the trade such that $n_i$ and $n_j$ satisfy,

$$0 = n_iD_{\$,i} + n_jD_{\$,j}$$

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Suppose you will use \\$1mm of capital, leveraged 50x to buy \\$50mm of the bonds (`204046`).

Use the ratio above to short a number of notes (`204047`) to keep zero duration.

Report the number of bonds and notes of your position, along with the total dollars in the short position.

## 1.3 Profit Opportunity

Using the concept of **modified duration**, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? Specifically, assume the convergence is symmetric: the bond's (`204046`) ytm goes down 0.5bp and the note (`204047`) ytm goes up 0.5bp.

Describe the PnL you would expect to achieve on your position should this happen. Specify the PnL of the long position, the short position, and the net total.

Suppose the spread in YTM between the two securities disappears, due to a symmetric move of roughly ~17bps in each security's YTM. What is the PnL? (This is just a linearly scaling of your prior answer for a 1bp convergence.) 


## 1.4 Result in 2008

Calculate the profit (or loss) on the position on the following two dates:
* 2008-11-25
* 2008-12-16

To calculate the pnl on each date, simply use the prices of the securities on those dates along with your position sizes, ($n_i, n_j$). No coupon is being paid in November or December, so all you need is the "dirty" price on these two dates.

Does the pnl make sense (approximately) given your results in 1.3 with regard to the sensitivity of pnl to moves in the YTM spread?

## 1.5 Examining the Trade through June 2009

Calculate the pnl of the trade for the following dates:
* 2009-01-27
* 2009-03-24
* 2009-06-16

Did the trade do well or poorly in the first six months of 2009?

Calculate the YTM spreads on these dates. Does the YTM spread correspond to pnl roughly as we would expect based on the calculation in 1.3?

***

# 2 Hedging Duration

Use data from `../data/treasury_ts_duration_2024-10-31.xlsx`.

The file contains time-series information on two treasuries. Observe the info of the securities with the following code:


In [54]:
# Import standard modules
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import sys
from pathlib import Path


# BASE_DIR = Path(__file__).absolute().parent.parent # Uncomment for python files
BASE_DIR = os.path.dirname(os.getcwd()) # Comment for python files
sys.path.insert(0, str(Path(BASE_DIR) / 'utils'))

import config
import scheffer_quant.fixed_income as fi

In [12]:
# Global variables
DATA_DIR = Path(config.DATA_DIR)
DATA_DIR.mkdir(parents=True, exist_ok=True)
QUOTE_DATE = '2024-10-31'
COMPOUNDING_FREQ = 2

FILE_PATH = Path(config.DATA_DIR) / f"treasury_ts_duration_{QUOTE_DATE}.xlsx"

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 100
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Extract data:
data = pd.read_excel(FILE_PATH, sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.6250
207392,bond,2019-08-15,2049-08-15,2.2500


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [14]:
price = pd.read_excel(FILE_PATH,sheet_name='price').set_index('quote date')
duration = pd.read_excel(FILE_PATH,sheet_name='duration').set_index('quote date')

display(price)
display(duration)

,207391,207392
quote date,,
2019-08-09,98.8828,99.7891
2019-08-12,99.7969,102.5547
2019-08-13,99.2812,101.8672
2019-08-14,100.4062,105.1797
2019-08-15,100.8828,106.2344
...,...,...
2024-11-22,88.7402,63.7227
2024-11-25,89.2871,65.3789
2024-11-26,89.2148,65.1758


,207391,207392
quote date,,
2019-08-09,9.2895,22.0001
2019-08-12,9.2855,22.1185
2019-08-13,9.2803,22.0843
2019-08-14,9.2828,22.2285
2019-08-15,9.2822,22.2709
...,...,...
2024-11-22,4.5394,17.2055
2024-11-25,4.5320,17.3123
2024-11-26,4.5291,17.2955


### 2.1.

Suppose you have a portfolio of `10,000` USD long in security `207391` on the first day of the sample.

If you want to manage interest rate exposure using duration, how large of a short position should you hold in `207392`?

In [22]:
SIZE = 10000
ID_LONG = 207391
ID_SHORT = 207392

In [27]:
first_date = duration.index[0]
short_position = SIZE * (duration.loc[first_date, ID_LONG] / duration.loc[first_date, ID_SHORT])
print(f'To hedge ${SIZE:,} in {ID_LONG}, short ${short_position:,.2f} in {ID_SHORT}.')

To hedge $10,000 in 207391, short $4,222.48 in 207392.


### 2.2.

Step through the time-series, doing the following:

* Starting at the end of the first day, set the hedged position according to the relative given durations.
* Use the second day's price data to evaluate the net profit or loss of the hedged position.
* Reset the the hedged position using the end-of-second-day durations. Again fix the long position of security `207391` to be `10,000`.
* Repeat throughout the timeseries.

Calculate the daily profit and loss (PnL) for the
* dynamically hedged position constructed above.
* long-only position, (still at `10,000` throughout.)

(You might check to verify that the net duration is zero at all dates.)

Report...
* the cumulative PnL of both strategies via a plot.
* the (daily) mean, standard deviation, min, and max of the PnL in a table.

In [ ]:
# Create position dataframe for the strategy
position = pd.DataFrame(index=duration.index, dtype=float)

position['long'] = SIZE / price[ID_LONG]
position['hedge ratio'] = (duration[ID_LONG] / duration[ID_SHORT]) * (price[ID_LONG] / price[ID_SHORT])
position['short'] = - position['hedge ratio'] * position['long']

In [ ]:
# Check that duration is indeed hedged as of end of day
position[['long ($)','short ($)']] = position[['long','short']] * price[[ID_LONG,ID_SHORT]].values
position['net ($)'] = position[['long ($)','short ($)']].sum(axis=1)
wts = position[['long ($)','short ($)']].div(position[['long ($)','short ($)']].sum(axis=1),axis=0)

position['duration'] = (wts * duration[[ID_LONG,ID_SHORT]].values).sum(axis=1)
position['duration'].describe().to_frame().T.style.format('{:.1%}')

,count,mean,std,min,25%,50%,75%,max
duration,133000.0%,0.0%,0.0%,-0.0%,-0.0%,0.0%,0.0%,0.0%


In [52]:
# P&L of the Hedged Position:
position['long p&l']  = position['long'].shift(1)  * price[ID_LONG].diff()
position['short p&l'] = position['short'].shift(1) * price[ID_SHORT].diff()
position['net hedged p&l']   = position['long p&l'] + position['short p&l']

In [50]:
# P&L of the Unhedged Position:
position['unhedged p&l'] = position['long'].shift(1) * price[ID_LONG].diff()

In [53]:
fig = px.line(position[['net hedged p&l','unhedged p&l']].cumsum(), title='Cumulative P&L of the Hedged Position')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Cumulative P&L ($)',
    yaxis=dict(
        tickformat=",0f",
        zeroline=True,
        zerolinecolor='darkgray', 
        zerolinewidth=1,
        gridcolor='lightgrey',
        showgrid=True
    )
)
fig.show()

### 2.3.

Give two reasons that the daily PnL is not always zero for the hedged position given that we have perfectly hedged the duration.

<span style="color:lightblue"> 

* The PnL is not zero because the two securities have different convexity:
    - Duration is only a linear approximation of the sensitivity of a bond to variation in the interest rates.
    - A portfolio hedged using only duration will experience residual gains or losses due to differences in convexity—especially during larger market moves.
* The PnL is not zero because the two securities have different coupon rates (and thus different cash flows).
    - Bonds with different coupons have different cash flow distributions, leading to differences in their reinvestment risk and price behavior over time. 
    - A higher-coupon bond, for instance, will distribute more cash earlier, which can lead to a shorter effective duration compared to a low-coupon bond.
    - Since bond prices incorporate expectations about future cash flows, differences in coupons will lead to mismatches in total returns over time, causing nonzero PnL even in a hedged position.

### 2.4.
The PnL above doesn't account for the coupons.

Calculate a dataframe indexed by dates with columns for the two treasuries with values of coupon payments. 
* Recall that the stated coupon rate is semiannual, so at any give coupon date, it pays half the stated rate.
* Figure out the coupon dates by using the `data` tab and looking for dates where `acc int` goes down. Recall that accrued interest measures the portion of the coupon period that has passed. So when this resets, it is because the coupon has been paid.

Report the first 5 dates that a coupon is paid (by either bond).

In [78]:
data[data['accrued int'] == 0].sort_values(by=['quote date', 'KYTREASNO'])

,KYTREASNO,type,quote date,issue date,maturity date,ttm,accrual fraction,cpn rate,bid,ask,price,accrued int,dirty price,ytm,duration
985,207391,note,2019-08-08,2019-08-15,2029-08-15,10.0205,0.0000,1.6250,99.1172,99.1797,99.1484,0.0000,99.1484,0.0171,9.2935
984,207391,note,2019-08-09,2019-08-15,2029-08-15,10.0178,0.0000,1.6250,98.8516,98.9141,98.8828,0.0000,98.8828,0.0174,9.2895
2100,207392,bond,2019-08-09,2019-08-15,2049-08-15,30.0178,0.0000,2.2500,99.7578,99.8203,99.7891,0.0000,99.7891,0.0226,22.0001
983,207391,note,2019-08-12,2019-08-15,2029-08-15,10.0096,0.0000,1.6250,99.7656,99.8281,99.7969,0.0000,99.7969,0.0165,9.2855
2285,207392,bond,2019-08-12,2019-08-15,2049-08-15,30.0096,0.0000,2.2500,102.5234,102.5859,102.5547,0.0000,102.5547,0.0213,22.1185
982,207391,note,2019-08-13,2019-08-15,2029-08-15,10.0068,0.0000,1.6250,99.2500,99.3125,99.2812,0.0000,99.2812,0.0170,9.2803
2328,207392,bond,2019-08-13,2019-08-15,2049-08-15,30.0068,0.0000,2.2500,101.8359,101.8984,101.8672,0.0000,101.8672,0.0216,22.0843
981,207391,note,2019-08-14,2019-08-15,2029-08-15,10.0041,0.0000,1.6250,100.3750,100.4375,100.4062,0.0000,100.4062,0.0158,9.2828
2618,207392,bond,2019-08-14,2019-08-15,2049-08-15,30.0041,0.0000,2.2500,105.1484,105.2109,105.1797,0.0000,105.1797,0.0202,22.2285
980,207391,note,2019-08-15,2019-08-15,2029-08-15,10.0014,0.0000,1.6250,100.8516,100.9141,100.8828,0.0000,100.8828,0.0153,9.2822


In [81]:
new_data = data[data['accrued int'] == 0].pivot_table(index='quote date',columns='KYTREASNO',values='accrued int',aggfunc='count')
new_data.resample('M').last().dropna(how='all')

KYTREASNO,207391,207392
quote date,,
2019-08-31,1.0000,1.0000
2022-02-28,1.0000,1.0000
2022-08-31,1.0000,1.0000
2023-02-28,1.0000,1.0000
2023-08-31,1.0000,1.0000
2024-02-29,1.0000,1.0000
2024-08-31,1.0000,1.0000


In [62]:
coupon_dates_long = fi.get_coupon_dates(quote_date=first_date, maturity_date=data_info.loc[ID_LONG, 'maturity date'])
coupon_dates_short = fi.get_coupon_dates(quote_date=first_date, maturity_date=data_info.loc[ID_SHORT, 'maturity date'])

coupons_long = pd.DataFrame(data_info.loc[ID_LONG, 'cpn rate']/2, columns=[ID_LONG], index=coupon_dates_long)
coupons_short = pd.DataFrame(data_info.loc[ID_SHORT, 'cpn rate']/2, columns=[ID_SHORT], index=coupon_dates_short)

coupons = pd.merge(coupons_long, coupons_short, how='outer', left_index=True, right_index=True)
coupons = coupons.fillna(0)
coupons


,207391,207392
2019-08-15,0.8125,1.1250
2020-02-15,0.8125,1.1250
2020-08-15,0.8125,1.1250
2021-02-15,0.8125,1.1250
2021-08-15,0.8125,1.1250
...,...,...
2047-08-15,0.0000,1.1250
2048-02-15,0.0000,1.1250
2048-08-15,0.0000,1.1250
2049-02-15,0.0000,1.1250


### 2.5.
Account for the coupons in the PnL calculations of `2.2`. Report the updated PnL in a plot and a table, similar to the reporting in `2.2`.

***